In [1]:
import os
import pandas as pd

In [39]:
seed = 2023

In [2]:
trn = pd.read_csv('./data/train_fillna.csv')
tst = pd.read_csv('./data/test_fillna.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [4]:
nor = trn.Delay.value_counts()[0] # not delay
abn = trn.Delay.value_counts()[1]

print((nor+abn)/len(trn))
print(abn/(nor+abn)) # abn 비율

0.255001
0.1764698961964855


In [8]:
trn.columns

Index(['Month', 'Day_of_Month', 'Origin_Airport', 'Origin_State',
       'Destination_Airport', 'Destination_State', 'Distance', 'Airline',
       'Tail_Number', 'Delay', 'Estimated_Departure_Hour',
       'Estimated_Departure_Min', 'Estimated_Arrival_Hour',
       'Estimated_Arrival_Min'],
      dtype='object')

In [15]:
str_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline','Tail_Number']
num_col = ['Month', 'Day_of_Month', 'Estimated_Departure_Hour', 'Estimated_Departure_Min','Estimated_Arrival_Hour', 'Estimated_Arrival_Min','Distance']

In [16]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
std = StandardScaler()
lab = LabelEncoder()

In [17]:
temp = trn.drop(columns=['Delay'])
for str in str_col:
    temp[str] = lab.fit_transform(temp[str])

for col in temp.columns.tolist():
    temp[[col]] = std.fit_transform(temp[[col]])

In [22]:
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from sklearn.metrics import f1_score

In [32]:
ecod = ECOD(contamination=(abn/(nor+abn)))
ecod.fit(temp)

copod = COPOD(contamination=(abn/(nor+abn)))
copod.fit(temp)

COPOD(contamination=0.1764698961964855, n_jobs=1)

In [33]:
temp_tst = tst.copy()
for str in str_col:
    temp_tst[str] = lab.fit_transform(temp_tst[str])

for col in temp_tst.columns.tolist():
    temp_tst[[col]] = std.fit_transform(temp_tst[[col]])

In [34]:
pred1 = ecod.predict(temp_tst)
pred2 = copod.predict(temp_tst)

In [35]:
pred_proba1 = ecod.predict_proba(temp_tst)
pred_proba2 = copod.predict_proba(temp_tst)

In [36]:
submission_temp = submission.copy()

In [37]:
submission_temp[['Not_Delayed', 'Delayed']] = pred_proba1
submission_temp

,ID,Not_Delayed,Delayed
0,TEST_000000,0.609041,0.390959
1,TEST_000001,0.755904,0.244096
2,TEST_000002,0.727835,0.272165
3,TEST_000003,0.726246,0.273754
4,TEST_000004,0.653701,0.346299
...,...,...,...
999995,TEST_999995,0.753032,0.246968
999996,TEST_999996,0.580964,0.419036
999997,TEST_999997,0.807398,0.192602
999998,TEST_999998,0.425605,0.574395


In [29]:
submission_temp.to_csv('./fillna_ecod.csv', index=False)

In [38]:
submission_temp[['Not_Delayed', 'Delayed']] = pred_proba2
submission_temp

,ID,Not_Delayed,Delayed
0,TEST_000000,0.717337,0.282663
1,TEST_000001,0.788568,0.211432
2,TEST_000002,0.843111,0.156889
3,TEST_000003,0.904540,0.095460
4,TEST_000004,0.811994,0.188006
...,...,...,...
999995,TEST_999995,0.878874,0.121126
999996,TEST_999996,0.670443,0.329557
999997,TEST_999997,0.828629,0.171371
999998,TEST_999998,0.496196,0.503804


In [40]:
submission_temp.to_csv('./fillna_copod.csv', index=False)

In [41]:
submission_temp[['Not_Delayed', 'Delayed']] = (pred_proba1+pred_proba2)/2
submission_temp

,ID,Not_Delayed,Delayed
0,TEST_000000,0.663189,0.336811
1,TEST_000001,0.772236,0.227764
2,TEST_000002,0.785473,0.214527
3,TEST_000003,0.815393,0.184607
4,TEST_000004,0.732847,0.267153
...,...,...,...
999995,TEST_999995,0.815953,0.184047
999996,TEST_999996,0.625703,0.374297
999997,TEST_999997,0.818013,0.181987
999998,TEST_999998,0.460900,0.539100


In [42]:
submission_temp.to_csv('./fillna_copod_ecod.csv', index=False)